In [8]:
#У нас две маленьких выборки.
#(Лучше вообще не сравнивать).
#Но если надо, используем валидацию t-testa, Mann-Whitneyu, бутстрапа.
#Многократно проводим на одной из выборок A/A тесты.
#Смотрим, какой метод работает "лучше" (показывает отсутствие различий).
#Этот способ должен быть самым крайним. Делать выводы таким образом опасно.
#Но все это лучше, чем на глазок.

import numpy as np
import random
from scipy import stats

#Валидация методов через A/A тест
#Валидируем t-test и Mann-Whitneyu
class Validator():
  def __init__(self, a, trials=100, alpha = 0.05):
    self.a = a
    self.trials = trials
    self.alpha = alpha
  
  def validate(self):
    self.validate_method(stats.mannwhitneyu)
    self.validate_method(stats.ttest_ind)

    permutator = Permutator(self.a, self.a, self.trials, self.alpha)  
    self.validate_method(permutator.evaluate_permutation_test(permutator.mean), permute=True)
    self.validate_method(permutator.evaluate_permutation_test(permutator.median), permute=True)

  #A/A тест для валидации метода
  def validate_method(self, method, permute=False):
    #Тестируем бутстрап
    #if (method.__name__ == 'mean') or (method.__name__ == 'median'):
    if (permute):
      #Первое А
      a = np.random.choice(self.a, len(self.a))
      #Второе А
      b = np.random.choice(self.a, len(self.a))
      inference = method
      p_value = inference[1]
      print(p_value)
      results = p_value
      p = self.p_test(results, method)

    #Тестируем все остальное (t-test и Mann-Whitneyu)
    else:
      results = []
      trial = 0
      while (trial < self.trials):
        a = np.random.choice(self.a, len(self.a))
        b = np.random.choice(self.a, len(self.a))
        if ( list(a) == list(b) ):
          continue
        else:
          inference = method(a, b)
          p_value = inference[1]
          if p_value >= self.alpha:
            results.append(0)
          else:
            results.append(1)
          trial += 1
      p = self.p_test(results, method)

  def p_test(self, results, method):
    if (isinstance(results, float)):
      p = results
      p = p * 100
      p = round(p, 2)
      if p == 0:
      #ноль может быть при очень малом количестве испытаний или когда ни одно не показало различий
        print(f'p is zero - %{p}')
        print(f'{method}: Invalid for usage')
        return p
      
      #валидный метод
      elif p < self.alpha:
        print(f'{method.__name__}: False positive rate is %{p}')
        print(f'{method}: Valid for usage')
        return p

      #А/A тест показывает различия - это плохо, метод не валидный
      else:
        print(f'{method.__name__}: False positive rate is too high - %{p}')
        print(f'{method}: is not recommended for usage' )
        return p
    
    else:
      p = results.count(1)/len(results) * 100
      p = round(p, 2)
      print(p)
      if p == 0:
        #ноль может быть при очень малом количестве испытаний или когда ни одно не показало различий
        p = 0.0001
        print(f'{method}: False positive rate is %{p}')
        print(f'{method}: Valid for usage')
        return p

      #валидный метод
      elif p < self.alpha:
        print(f'{method.__name__}: False positive rate is %{p}')
        print(f'{method}: Valid for usage')
        return p

      #А/A тест показывает различия - это плохо, метод не валидный
      else:
        print(f'{method}: False positive rate is too high - %{p}')
        print(f'{method}: is not recommended for usage' )
        return p

#Бутстрап - сравниваем
#ЗАДАНИЕ: объединить методы в один, так как паттерн повторяется 
class Permutator():
  def __init__(self, a, b, trials=100, alpha = 0.05):
    self.a = a
    self.b = b
    self.trials = trials
    self.alpha = alpha
  
  def mean(self, i):
    return np.mean(i)
    
  def median(self, i):
    return np.median(i)

  def get_difference(self, t_statistics_method):
    central_tendency_a = t_statistics_method(self.a)
    central_tendency_b = t_statistics_method(self.b)
    diff_observed = central_tendency_a - central_tendency_b
    diff_observed = abs(diff_observed)
    return diff_observed

  def evaluate_permutation_test(self, t_statistics_method):
    diff_observed = self.get_difference(t_statistics_method)
    results = []
    trial = 0
    combination = self.a + self.b
    while (trial < self.trials):
      shuffled = combination
      if (len(shuffled)%2 != 0):
        index = random.randint(0, len(shuffled)-1)
        shuffled = np.delete(shuffled, index)

      np.random.shuffle(shuffled)
      a = shuffled[:len(self.a)]
      b = shuffled[-len(self.b):]

      diff_permutated = self.get_difference(t_statistics_method)
      if (diff_permutated > diff_observed):
        results.append(1)
      else:
        results.append(0)
      trial += 1
    
    p_stats = [self.alpha, results.count(1)/len(results)]
    return p_stats

In [11]:
#Data 1 vs Data 2
a = [24, 661, 707, 106, 71, 262]
b = [125, 225, 608, 402]
validator = Validator(a, 10000, 0.05)
validator.validate()

8.48
<function mannwhitneyu at 0x7f2157b60a70>: False positive rate is too high - %8.48
<function mannwhitneyu at 0x7f2157b60a70>: is not recommended for usage
5.3
<function ttest_ind at 0x7f2157b5e3b0>: False positive rate is too high - %5.3
<function ttest_ind at 0x7f2157b5e3b0>: is not recommended for usage
0.0
p is zero - %0.0
[0.05, 0.0]: Invalid for usage
0.0
p is zero - %0.0
[0.05, 0.0]: Invalid for usage


In [12]:
permutator = Permutator(a, b, 10000, 0.05)
print(stats.mannwhitneyu(a, b))
print(stats.ttest_ind(a, b))
print(permutator.evaluate_permutation_test(permutation.mean))
print(permutator.evaluate_permutation_test(permutation.median))

MannwhitneyuResult(statistic=10.0, pvalue=0.37455956650029765)
Ttest_indResult(statistic=-0.19727328908358727, pvalue=0.8485336476483601)
[0.05, 0.0]
[0.05, 0.0]
